In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import pandas as pd
3 in pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]}).index

False

In [13]:
from mapmanagercore.lazy_geo_pandas import schema, calculated, LazyGeoFrame, LazyGeoSeries, seriesSchema

@seriesSchema()
class Config:
    t: int = 1
    b: int = 5
    c: int = 6

@schema(index=["b", "t"])
class B:
    t: int = 1
    b: int = 5
    c: int = 6


@schema(index=["a", "t"], relationships={
    B: ["b", 't']
}, properties={
    "a": {
        "categorical": False,
        "title": "A"
    },
    "t": {
        "categorical": True,
        "title": "Time"
    }
})
class A:
    a: int = 3
    t: int
    b: int

    @calculated(title="Test", dependencies=["b", "c"])
    def calculatedTest(frame: LazyGeoFrame):
        return frame["b"]

    @calculated(title="Test2", dependencies={"A": ["calculatedTest"], "Config": ["b"]})
    def calculatedTest2(frame: LazyGeoFrame):
        config = frame.getFrame("Config")
        return config["b"]


aFrame = LazyGeoFrame(A)
bFrame = LazyGeoFrame(B)
cSeries = LazyGeoSeries(Config)
aFrame.update((1, 2), A(b=2))
aFrame.update((2, 2), A(b=2))
bFrame.update((2, 2), B(c=3))

In [20]:
cSeries.update(Config.withDefaults(b=3))

In [8]:
cSeries[["b", "c"]]

b    2
c    6
Name: 0, dtype: Int64

In [4]:
aFrame.redo()

In [11]:
aFrame.update((1, 2), A(a=1, t=2, b=3))


In [21]:
aFrame[:]

,,b,calculatedTest,calculatedTest2
a,t,,,
1,2,2,2,3.0
2,2,2,2,3.0


In [22]:
aFrame._rootDf

,,b,modified,calculatedTest,calculatedTest.valid,calculatedTest2,calculatedTest2.valid
a,t,,,,,,
1,2,2,2024-05-15 14:03:41.482096,2,True,3.0,True
2,2,2,2024-05-15 14:03:41.485977,2,True,3.0,True
